In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

%matplotlib inline
np.random.seed(1)

### 1. Load and prepare dataset

In [ ]:
# Load training data
train_dataset = h5py.File('train_catvnoncat.h5', "r")
train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

In [ ]:
# Load test data
test_dataset = h5py.File('test_catvnoncat.h5', "r")
test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels

In [ ]:
# Check all the classes
classes = np.array(test_dataset["list_classes"][:])

In [ ]:
classes

In [ ]:
# Reshape the train and test set labels
train_set_y = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
test_set_y = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

In [ ]:
# Example of a picture
index = 50
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

In [ ]:
# Checking the shape of datasets

m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")

print ("train_set_x_orig shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_orig shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
# reshape
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
# Standardizing dataset
train_set_x = train_set_x_flatten/255
test_set_x = test_set_x_flatten/255

### 2. Model 

In [ ]:
# Sigmoid function
def sigmoid(Z):
    A = 1 / (1+np.exp(-Z))
    return A

In [ ]:
# ReLU function
def relu(Z):
    A = np.maximum(0,Z)
    return A

In [ ]:
# Initializing model parameters W and b
def init_params(layer_dims):
    np.random.seed(3)
    params = {}
    L = len(layer_dims)
    for l in range(1, L):
        params['W'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])/ np.sqrt(layer_dims[l-1])
        params['b'+str(l)] = np.zeros((layer_dims[l], 1))
    
    return params

In [ ]:
# Forward propagation step (for 1 layer)
def forward_prop_step(A_prev, W, b, activ_func):
    Z = np.dot(W, A_prev) + b
    
    if activ_func == 'relu':
        A = relu(Z)
    elif activ_func == 'sigmoid':
        A = sigmoid(Z)

    cache = (A_prev, W, b, Z)    
    return A, cache

In [ ]:
# Forward propagation - full iteration (for 1 to L layers)
def forward_prop_model(X, params): 
    caches=[]
    L = len(params)//2
    A = X
    
    # Apply ReLU activaition function, while propagating forward over the layers from 1 to L-1
    for l in range (1,L):
        A_prev = A
        A, cache = forward_prop_step(A_prev, params['W'+str(l)], params['b'+str(l)], 'relu')
        caches.append(cache)
    
    # Apply Sigmoid activaition function, while propagating forward over the last layer (layer L)
    AL, cache = forward_prop_step(A, params['W'+str(L)], params['b'+str(L)], 'sigmoid')
    caches.append(cache)
    
    return AL, caches

In [ ]:
# Cost function
def cost_func(A, Y):
    m = Y.shape[1]    
    cost = -1/m * np.sum(np.multiply(Y, np.log(A)) + np.multiply(1-Y, np.log(1-A)))
    return cost

In [ ]:
# Calculating dZ for backpropagation step for the layer with sigmoid
def sigmoid_backward(dA, Z):
    g_deriv = np.multiply (sigmoid(Z), (1-sigmoid(Z)))
    dZ = np.multiply (dA, g_deriv)
    return dZ

In [ ]:
# Calculating dZ for backpropagation step for the layer with ReLU
def relu_backward(dA, Z):
    g_deriv = np.where(Z<0, 0, 1)    
    dZ = np.multiply(dA, g_deriv)
    return dZ

In [ ]:
# Backpropagation step (for 1 layer)
def back_prop(dA, cache, activ_func):
    m = dA.shape[1]
    
    A_prev, W, b, Z = cache
    
    if activ_func == 'relu':
        dZ = relu_backward(dA, Z)
    elif activ_func == 'sigmoid':
        dZ = sigmoid_backward(dA, Z)

    dW = 1/m * np.dot(dZ, A_prev.T)
    db = 1/m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dW, db, dA_prev

In [ ]:
# Backpropagation - full iteration (for L to 1 layers)
def back_prop_model (AL, Y, caches):    # for whole NN (all layers) 
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # Backpropagating over the last layer (layer L) with Sigmoid activaition function
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))        
    current_cache = caches[L-1]    
    grads['dW'+str(L)], grads['db'+str(L)], grads['dA'+str(L-1)] = back_prop(dAL, current_cache, 'sigmoid')    
    
    # Backpropagating over the layers from L-1 to 1 with ReLU activaition function
    for l in reversed(range(L-1)):  # from l=L-2 to l=0
        current_cache = caches[l]
        dA = grads['dA'+str(l+1)]
        grads['dW'+str(l+1)], grads['db'+str(l+1)], grads['dA'+str(l)] = back_prop(dA, current_cache, 'relu')
        
    return grads

In [ ]:
# Updating parameters W and b
def update_params(params, grads, learning_rate):
    L = len(params)//2
    for l in range(L):
        params['W'+str(l+1)] = params['W'+str(l+1)] - learning_rate * grads['dW'+str(l+1)]
        params['b'+str(l+1)] = params['b'+str(l+1)] - learning_rate * grads['db'+str(l+1)]

    return params

In [ ]:
# L-layers NN model
def model_L_Layers(X, Y, layer_dims, num_iter = 2500, learning_rate = 0.0075):

    parameters = init_params(layer_dims)
    
    for i in range(0, num_iter):
        # Forward propagation
        AL, caches = forward_prop_model (X, parameters)
                
        #Cost-function
        cost = cost_func(AL, Y)
        
        # Backpropagation
        grads = back_prop_model (AL, Y, caches)
        
        # Updating parameters W and b
        parameters = update_params(parameters, grads, learning_rate)     

        # print Cost-function on every 100-th iteration
        if i%100 == 0:    
            print ("Cost after iteration %i: %f" %(i, cost))    
            
    return parameters

In [ ]:
# Building the model. As a result - get its parameters W and b.

# Set the width and depth of the model 
n_x = train_set_x.shape[0]   #  input layer (layer 0)
layer_dims = [n_x, 20, 7, 5, 1] #  4-layer model

parameters = model_L_Layers(train_set_x, train_set_y, layer_dims, num_iter = 3000, learning_rate = 0.0075)

### 3. Prediction

In [ ]:
# Predicting Y, having W and b parameters
def predict_Y (X, params):
    m = X.shape[1]
    Y_predict = np.zeros((1, m))

    AL, caches = forward_prop_model(X, params)

    for i in range(m):
        # Convert probabilities A[0,i] to actual predictions p[0,i]    
        if AL[0,i] > 0.5:
            Y_predict[0,i] = 1
    return Y_predict

In [ ]:
# Predicting Y on train and test datasets
Y_train_prediction = predict_Y (train_set_x, parameters)
Y_test_prediction = predict_Y (test_set_x, parameters)
    
# Print train and test accuracy
print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_train_prediction - train_set_y)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_test_prediction - test_set_y)) * 100))